# 1 - Imports

In [1]:
import sys
sys.path.append('/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/scripts/')

import argparse
import librosa
import librosa.display as ld
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from model_evaluator_2 import ModelEvaluator
from settings import MODEL_EVALUATOR_DEFAULT_SETTINGS

# 2 - Feature extractor class

In [2]:
default_params_dict = MODEL_EVALUATOR_DEFAULT_SETTINGS

default_params_dict['model_checkpoint_path'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/models/v1_tiny/v1_tiny_VGGNL_DoubleMHA_700.chkpt'
default_params_dict['test_clients'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/test/tiny_check/clients.ndx'
default_params_dict['test_impostors'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/test/tiny_check/impostors.ndx'
default_params_dict['dump_folder'] = './models_results/v1___'
default_params_dict['data_dir'] = ['/home/usuaris/veussd/DATABASES/VoxCeleb/VoxCeleb1/test/']

default_params = argparse.Namespace(**default_params_dict)

model_evaluator_2 = ModelEvaluator(default_params)

22-10-06 22:19:24 - model_evaluator_2 - INFO - Setting device...
22-10-06 22:19:24 - model_evaluator_2 - INFO - Running on cuda device.
22-10-06 22:19:24 - model_evaluator_2 - INFO - Device setted.
22-10-06 22:19:24 - model_evaluator_2 - INFO - Setting random seed...
22-10-06 22:19:24 - model_evaluator_2 - INFO - Random seed setted.
22-10-06 22:19:24 - model_evaluator_2 - INFO - Loading checkpoint from /home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/models/v1_tiny/v1_tiny_VGGNL_DoubleMHA_700.chkpt
22-10-06 22:19:25 - model_evaluator_2 - INFO - Model checkpoint was saved at epoch 0
22-10-06 22:19:25 - model_evaluator_2 - INFO - Checkpoint loaded.


# 3 - Tests

In [3]:
model_evaluator_2.load_data()
model_evaluator_2.total_batches = len(model_evaluator_2.training_generator)

22-10-06 22:19:26 - model_evaluator_2 - INFO - Loading data from /home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/test/tiny_check/clients.ndx
22-10-06 22:19:26 - model_evaluator_2 - INFO - Loading data from /home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/test/tiny_check/impostors.ndx
/home/usuaris/veu/federico.costa/.conda/envs/DASV/lib/python3.10/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
22-10-06 22:19:26 - model_evaluator_2 - INFO - Data and labels loaded.


In [92]:
with torch.no_grad():

    # Switch torch to evaluation mode
    model_evaluator_2.net.eval()

    for model_evaluator_2.batch_number, (input_1, len_1, input_2, len_2, label) in enumerate(model_evaluator_2.training_generator):

        print(f"Batch {model_evaluator_2.batch_number} of {model_evaluator_2.total_batches}")

        input_1 = input_1.float().to(model_evaluator_2.device)
        print(input_1.size())
        input_2 = input_2.float().to(model_evaluator_2.device)
        len_1 = len_1.int().to(model_evaluator_2.device)
        len_2 = len_2.int().to(model_evaluator_2.device)
        label = label.int().to(model_evaluator_2.device)
        
        input_1 = [tensor[:len_1[i],:].unsqueeze(0) for i, tensor in enumerate(input_1)]
        input_2 = [tensor[:len_2[i],:].unsqueeze(0) for i, tensor in enumerate(input_2)]
        
        if torch.cuda.device_count() > 1:
            embedding_1 = [model_evaluator_2.net.module.get_embedding(input) for input in input_1]
            embedding_2 = [model_evaluator_2.net.module.get_embedding(input) for input in input_2]
        else:
            embedding_1 = [model_evaluator_2.net.get_embedding(input) for input in input_1]
            embedding_2 = [model_evaluator_2.net.get_embedding(input) for input in input_2]

        break

Batch 0 of 9
torch.Size([4, 482, 80])


In [93]:
embedding_1.size()

AttributeError: 'list' object has no attribute 'size'

In [98]:
from utils import scoreCosineDistance
dist = [scoreCosineDistance(emb_1, emb_2) for emb_1, emb_2 in zip(embedding_1, embedding_2)][:5]

In [101]:
torch.tensor(dist)

tensor([0.7450, 0.5279, 0.5160, 0.3977])

In [102]:
list(zip(torch.tensor(dist).cpu().detach().numpy(), label.cpu().detach().numpy()))

[(0.74496925, 1), (0.5279028, 1), (0.5159511, 1), (0.39771855, 1)]

In [87]:
input_1[0].size()

torch.Size([482, 80])

In [90]:
model_evaluator_2.net.get_embedding(input_1[0].unsqueeze(0))

tensor([[-4.9273e-02, -2.5617e-01, -2.2962e-01, -6.8943e-01, -5.4523e-01,
         -5.4669e-01, -6.1277e-01, -4.5274e-01,  8.0051e-01, -5.8577e-01,
          9.8880e-01, -7.6463e-01, -5.8092e-01, -1.6867e-01, -1.6307e-01,
         -3.6484e-01, -8.2553e-01, -6.9187e-01,  1.0534e+00, -5.6010e-01,
         -7.5977e-01,  1.6055e+00, -3.0820e-01, -6.6317e-01, -5.2201e-02,
         -5.3053e-01, -6.8584e-01,  4.0441e-01, -7.2524e-01, -8.2029e-01,
         -6.7179e-01, -7.2336e-01, -6.3542e-01,  7.0891e-01, -2.9244e-01,
         -6.0293e-01, -8.1141e-01, -3.2447e-01, -5.9938e-01, -8.2280e-01,
          5.0942e-01,  9.3585e-01,  1.4759e-01, -5.7271e-01, -5.8872e-01,
         -7.5093e-01, -7.6130e-01, -6.9583e-01, -7.6415e-01,  4.1675e-02,
         -6.5178e-01,  9.7255e-02, -5.9963e-01,  1.2134e+00,  7.4425e-01,
         -3.1699e-01, -7.6056e-01,  9.1079e-03, -5.6457e-01,  7.4538e-01,
         -8.3574e-01, -4.8584e-01,  9.1115e-01, -6.0348e-01,  2.7966e-01,
         -7.0498e-01,  1.3194e+00,  2.

In [80]:
torch.cat(l).size()

torch.Size([2352, 80])

In [77]:
torch.tensor(l)

ValueError: only one element tensors can be converted to Python scalars

In [76]:
np.array(l)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [71]:
torch.as_tensor(np.array(l))

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [19]:
input_2[:,:len_2,:].size()

TypeError: only integer tensors of a single element can be converted to an index

In [24]:
input_2[0][:,:] >= len_2[0]

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [33]:
len_2

tensor([498, 850, 542, 462])

In [45]:
seq_lens = torch.tensor([2,3,5,1]).unsqueeze(-1)
seq_lens

tensor([[2],
        [3],
        [5],
        [1]])

In [42]:
seq_lens = torch.tensor([2,3,5,1]).unsqueeze(-1)
max_len = torch.max(seq_lens)

# create tensor of suitable shape and same number of dimensions
range_tensor = torch.arange(max_len).unsqueeze(0)
range_tensor

tensor([[0, 1, 2, 3, 4]])

In [44]:
seq_lens = torch.tensor([2,3,5,1]).unsqueeze(-1)
max_len = torch.max(seq_lens)

# create tensor of suitable shape and same number of dimensions
range_tensor = torch.arange(max_len).unsqueeze(0)
range_tensor = range_tensor.expand(seq_lens.size(0), range_tensor.size(1))
range_tensor

tensor([[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]])

In [36]:
seq_lens = torch.tensor([2,3,5,1]).unsqueeze(-1)
max_len = torch.max(seq_lens)

# create tensor of suitable shape and same number of dimensions
range_tensor = torch.arange(max_len).unsqueeze(0)
range_tensor = range_tensor.expand(seq_lens.size(0), range_tensor.size(1))

# until this step, we only created auxiliary tensors (you may already have from previous steps) 
# the real mask tensor is created with binary masking:
mask_tensor = (range_tensor < seq_lens) 

In [47]:
range_tensor

tensor([[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]])

In [46]:
seq_lens

tensor([[2],
        [3],
        [5],
        [1]])

In [37]:
mask_tensor

tensor([[ True,  True, False, False, False],
        [ True,  True,  True, False, False],
        [ True,  True,  True,  True,  True],
        [ True, False, False, False, False]])

In [53]:
torch.arange(10).view(2,5) < 4

tensor([[ True,  True,  True,  True, False],
        [False, False, False, False, False]])

In [17]:
input_2[0][:len_2[0], :].size()

torch.Size([498, 80])

In [ ]:
len_1

In [ ]:
input_2.size()

In [ ]:
len_2

In [ ]:
labels

In [ ]:
1/0

In [ ]:
from data import normalizeFeatures, featureReader, TestDataset

with open(model_evaluator_2.input_params.test_clients, 'r') as clients_file:
    clients_trials = clients_file.readlines()

# Read the paths of the impostors audios
with open(model_evaluator_2.input_params.test_impostors, 'r') as impostors_file:
    impostors_trials = impostors_file.readlines()

# Instanciate a Dataset class
dataset = TestDataset(clients_trials, impostors_trials, model_evaluator_2.params, model_evaluator_2.input_params, model_evaluator_2.net)

In [ ]:
dataset[0]

In [ ]:
input_1.size()

In [ ]:
label.size()

In [ ]:
len_1

In [ ]:
a = np.array([[1,2,3],[1,2,4]])
a.shape

In [ ]:
torch.from_numpy(a)

In [ ]:
torch.tensor(1)

In [ ]:
input_1.size()

In [ ]:
len_1

In [ ]:
label.size()

In [ ]:
label[0].item()

In [ ]:
tensor_example_1 = torch.cat([input_1[0], torch.ones(14, 80).to(model_evaluator_2.device)])
tensor_example_2 = torch.cat([input_1[1], torch.ones(23, 80).to(model_evaluator_2.device)])
tensor_example_3 = torch.cat([input_1[2], torch.ones(11, 80).to(model_evaluator_2.device)])

data = [
    (tensor_example_1, label[0].item(), tensor_example_1.size(0)), 
    (tensor_example_2, label[1].item(), tensor_example_2.size(0)),
    (tensor_example_3, label[2].item(), tensor_example_3.size(0)),
]

features, labels, lengths = zip(*data)

max_len = max(lengths)
# We are assuming that all features have the same number of columns
number_columns = data[0][0].size(1)
padded_features = torch.zeros((len(data), max_len, number_columns))

for i in range(len(data)):
    features_tensor = data[i][0]
    rows, cols = features_tensor.size(0), features_tensor.size(1)
    padded_features_tensor = torch.cat(
        [
            features_tensor, 
            torch.zeros((max_len - rows, cols)).to(model_evaluator_2.device)
        ]
    )
    
    padded_features[i] = padded_features_tensor
    
labels = torch.tensor(labels)
lengths = torch.tensor(lengths)
    
padded_features.size()

In [ ]:
max_len

In [ ]:
lengths

In [ ]:
rows

In [ ]:
a = torch.cat([input_1[0], torch.ones(14, 80).to(model_evaluator_2.device)])
a.size()

In [ ]:
features[0][0]

In [ ]:
a = [torch.tensor([1,2,3]), torch.tensor([3,4])]
b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)
b



In [ ]:
torch.nn.utils.rnn.pack_padded_sequence(b, batch_first=True, lengths=[3,2])

In [ ]:
features